In [22]:
import requests
import pandas as pd
import xlsxwriter
from datetime import datetime

In [23]:
#base URL
base_url = 'https://legislation.nysenate.gov/api/3/bills/search'

#API key info. need to change into secret variable
api_key = "XLC9cIQDZOgmCiMCT43D1umf6VeOFuGU"



In [24]:
def fetch_bill_details(api_key, search_term, session_year):
    url = f"{base_url}?term={search_term}&session_year={session_year}&key={api_key}"
    headers = {'Authorization': f'Token {api_key}'}

    try:
        response = requests.get(url, headers=headers)
        print(f"Response Status Code: {response.status_code}")
        print(f"Response Content: {response.text}")
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error getting data: {e}")
        return None

In [25]:
def flatten_bill_data(data, prefix=""):  # More general flattening function
    flattened = {}
    for key, value in data.items():
        new_key = f"{prefix}_{key}" if prefix else key
        if isinstance(value, dict):
            flattened.update(flatten_bill_data(value, new_key))  # Recursive call
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    flattened.update(flatten_bill_data(item, f"{new_key}_{i}")) #Flatten each item in the list
                else:
                    flattened[f"{new_key}_{i}"] = item
        else:
            flattened[new_key] = value
    return flattened

In [26]:
def export_to_excel(bill_data, filename_prefix="NY_Senate_Bills_data"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.xlsx"

    if 'result' in bill_data and 'items' in bill_data['result']:
        items = bill_data['result']['items']
        flattened_items = []  # List to store flattened bill data

        for item in items:  # Flatten each bill item
            flattened_data = flatten_bill_data(item)
            flattened_items.append(flattened_data)

        df = pd.DataFrame(flattened_items)  # Create DataFrame from the list of flattened dictionaries

        if not df.empty:
            with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
                df.to_excel(writer, index=False, sheet_name='Bills')
                print(f"Data has been exported to {filename}")
        else:
            print("No data to export to Excel.")
    else:
        print("'result.items' not found in the API response or is empty.")

In [27]:
#session year and search term
session_year = 2024
search_term = "vote"  # Replace search term/terms

In [28]:
# export bill details
bill_data = fetch_bill_details(api_key, search_term, session_year)

if bill_data:
    export_to_excel(bill_data)
else:
    print("No data retrieved from the API.")

Response Status Code: 200
Response Content: {
  "success" : true,
  "message" : "",
  "responseType" : "search-results list",
  "total" : 10840,
  "offsetStart" : 1,
  "offsetEnd" : 25,
  "limit" : 25,
  "result" : {
    "items" : [ {
      "result" : {
        "basePrintNo" : "S7138",
        "session" : 2009,
        "basePrintNoStr" : "S7138-2009",
        "printNo" : "S7138",
        "billType" : {
          "chamber" : "SENATE",
          "desc" : "Senate",
          "resolution" : false
        },
        "title" : "Creates the economic benefit power program",
        "activeVersion" : "",
        "year" : 0,
        "publishedDateTime" : "2010-03-16T10:56:53",
        "substitutedBy" : null,
        "sponsor" : {
          "member" : {
            "memberId" : 451,
            "chamber" : "SENATE",
            "incumbent" : false,
            "fullName" : "Darrel J. Aubertine",
            "shortName" : "AUBERTINE",
            "imgName" : "Darrel J. Aubertine.jpg",
            